In [ ]:
import os

os.environ["HOME"] = os.path.expanduser("~")

# Read Clarifai API key from environment. Export CLARIFAI_API_KEY before running this notebook.
CLARIFAI_API_KEY = os.environ.get('CLARIFAI_API_KEY')
if not CLARIFAI_API_KEY:
    print("Warning: CLARIFAI_API_KEY not set in environment.")

# Make data

this part takes the facial data points from SCUT and makes face_measurements.csv 

In [ ]:
# Extract facial geometry from SCUT-FBP5500 landmarks in .pts format

import os
import csv
import math

# === Helper: Euclidean distance ===
def distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# === Parse .pts landmark file with encoding fallback ===
def parse_pts(filepath):
    import numpy as np

    try:
        with open(filepath, 'rb') as f:
            raw = f.read()
        float_data = np.frombuffer(raw, dtype=np.float32)
    except Exception as e:
        raise ValueError(f"Binary load failed: {e}")

    if len(float_data) != 173:
        raise ValueError(f"Unexpected float count: {len(float_data)}")

    coords = float_data[1:]  # skip first float (junk/header)
    landmarks = list(zip(coords[::2], coords[1::2]))  # (x, y) pairs
    return landmarks

# === Extract facial metrics ===
def compute_metrics(landmarks):
    chin = landmarks[55]         # bottom of chin
    forehead = landmarks[19]     # top center
    jaw_left = landmarks[0]      # left jaw corner
    jaw_right = landmarks[10]    # right jaw corner
    eye_left = landmarks[37]     # left pupil center
    eye_right = landmarks[46]    # right pupil center

    width = distance(jaw_left, jaw_right)
    height = distance(forehead, chin)
    eye_spacing = distance(eye_left, eye_right)
    ratio = width / height if height else 0

    return width, height, ratio, eye_spacing

# === Main loop ===
landmark_dir = "SCUT-FBP5500/landmarks"  # Update path as needed
output_csv = "face_measurements.csv"
count = 0

with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["image", "face_width", "face_height", "width_to_height", "eye_spacing"])

    for filename in os.listdir(landmark_dir):
        if filename.endswith(".pts"):
            filepath = os.path.join(landmark_dir, filename)
            try:
                landmarks = parse_pts(filepath)
                if len(landmarks) != 86:
                    print(f"⚠️ Skipped {filename} (not 86 points)")
                    continue
                width, height, ratio, eye_spacing = compute_metrics(landmarks)
                img_name = filename.replace(".pts", ".jpg")
                writer.writerow([img_name, width, height, ratio, eye_spacing])
                count += 1
            except Exception as e:
                print(f"❌ Error processing {filename}: {str(e) or type(e).__name__}")

print(f"✅ Processed {count} landmark files into {output_csv}")


take face_measurements.csv and make it a pandas dataframe with the beauty scores

In [ ]:
import pandas as pd

data = pd.read_csv('face_measurements.csv')

In [ ]:
data

# Make recc logic

rule based using ehrustics

In [ ]:
def recommend_glasses(face_width, face_height, eye_spacing):
    """
    Recommends glasses styles based on facial measurements.
    
    Parameters:
        face_width (float): Distance between jaw corners
        face_height (float): Distance from forehead to chin
        eye_spacing (float): Distance between eye centers
        
    Returns:
        List of (style, score) tuples, sorted by score descending
    """
    recommendations = []

    width_to_height = face_width / face_height if face_height else 0

    # Rule: Wide faces -> soften with round or oversized
    if width_to_height > 1.5:
        recommendations.append(("Round", 0.9))
        recommendations.append(("Oversized", 0.8))

    # Rule: Long/narrow faces -> add width with square or bold frames
    if face_height > face_width * 1.3:
        recommendations.append(("Square", 0.85))
        recommendations.append(("Cat-eye", 0.75))

    # Rule: Close-set eyes -> minimal bridge
    if eye_spacing < face_width * 0.25:
        recommendations.append(("Rimless", 0.8))

    # Rule: Wide-set eyes -> bold bridge styles
    if eye_spacing > face_width * 0.35:
        recommendations.append(("Rectangle", 0.8))

    # Rule: Average proportions → fallback recommendation
    if not recommendations:
        recommendations.append(("Rectangle", 0.7))
        recommendations.append(("Round", 0.6))

    # Sort by score descending
    sorted_recs = sorted(recommendations, key=lambda x: -x[1])
    return sorted_recs



reccs = []
for i in range(len(data)):
    reccs.append(recommend_glasses(data.iloc[i].face_width, data.iloc[i].face_height, data.iloc[i].eye_spacing))
